Source code for our research ....

The main code include two parts, which are Cloud model-DEMATEL and Fuzzy based PROMETHEE II.

In [28]:
# load modules
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings("ignore")

## Cloud model-DEMATEL

### Define the Convert_LIT_to_CloudModel function to convert user evaluation linguistic to cloud model

In [29]:
def Convert_LIT_to_CloudModel(assessment_matrix):
    '''
    input:
        evaluation_matrix: linguistic term assessment matrix provided by user,
                           size: n×n
    output:
        C_Ex: the expectation of cloud model, size: n×n
        C_En: the entropy of cloud model, size: n×n
        C_He: the super-entropy of cloud model, size: n×n
    Reference: Identifying critical causal criteria of green supplier evaluation
               using heterogeneous judgements: An integrated approach based on cloud
               model and DEMATEL
    '''
    # 有效域:[X_min,X_max] = [0,10]
    # 语言项集：S = {s_-2,s_-1,s_0,s_1,s_2}
    linguistic_term_table = {'s-2':(0,2.85,0.16),
                             's-1':(2.92,2.45,0.29),
                             's0': (5,2.13,0.40),
                             's1': (7.08,2.45,0.29),
                             's2': (10,2.85,0.16)}
    C_Ex = np.zeros(assessment_matrix.shape)
    C_En = np.zeros(assessment_matrix.shape)
    C_He = np.zeros(assessment_matrix.shape)
    for i in range(assessment_matrix.shape[0]):
        for j in range(assessment_matrix.shape[1]):
            if i == j:
                C_Ex[i][j] = 0
                C_En[i][j] = 0
                C_He[i][j] = 0
            else:
                assessment_value_ij = assessment_matrix[i][j]
                cloud_model_ij = linguistic_term_table[assessment_value_ij]
                C_Ex[i][j] = cloud_model_ij[0]
                C_En[i][j] = cloud_model_ij[1]
                C_He[i][j] = cloud_model_ij[2]
    return C_Ex,C_En,C_He

### Weight calculation methods of DEMATEL

#### Method proposed by Cebi (2013)

Reference: Cebi, S. (2013). **Determining importance degrees of website design parameters based on interactions and types of websites**. Decision Support Systems, 54(2), 1030-1043. https://doi.org/10.1016/j.dss.2012.10.036

In [30]:
def weight_by_Cebi_method(P_L,P_U):
    P_mean = (P_L + P_U)/2
    weight = P_mean/sum(P_mean)
    return np.around(weight,4)

#### Method proposed by Wang (2015)

Reference: Wang, C. (2015).**Using quality function deployment to conduct vendor assessment and supplier recommendation for business-intelligence systems.** Computers & Industrial Engineering,84,24-31.https://doi.org/10.1016/j.cie.2014.10.005

In [31]:
def weight_by_Wang_method(R_L,R_U):
    abs_R = np.abs((R_L+R_U)/2)
    weight = abs_R/sum(abs_R)
    return np.around(weight,4)

#### Method proposed by Zhou et al. (2023)

Reference: Zhou,T.,et al. (2023). **Smart experience-oriented customer requirement analysis for smart product service system A novel hesitant fuzzy linguistic cloud DEMATEL method.** Advanced Engineering Informatics, 56, 101917.https://doi.org/10.1016/j.aei.2023.101917

In [32]:
def weight_by_Zhou_method(P_L,P_U,P_Ex,R_L,R_U,R_Ex):
    PL_REx = np.power(P_L,2) + np.power(R_Ex,2)
    PU_REx = np.power(P_U,2) + np.power(R_Ex,2)
    RL_PEx = np.power(R_L,2) + np.power(P_Ex,2)
    RU_PEx = np.power(R_U,2) + np.power(P_Ex,2)
    L = (np.sqrt(PL_REx) + np.sqrt(PU_REx) + np.sqrt(RL_PEx) + np.sqrt(RU_PEx))/4
    weight = L/sum(L)
    return np.around(weight,4)

#### Method proposed in this paper

In [33]:
def weight_by_our_method(P_L,P_U,R_L,R_U):
    combined_mean_value = (P_L + P_U + R_L + R_U)/4
    weight = combined_mean_value/sum(combined_mean_value)
    return np.around(weight,4)

### Define the Cloud_DEMATEL function to calculate the preference weight of user

In [36]:
def get_ranking(input_values):
    idx = np.argsort(-input_values)
    ranking = np.zeros(input_values.shape)
    for i in range(input_values.shape[0]):
        ranking[idx[i]] = i+1
    return ranking

def Cloud_DEMATEL(input=None):
    # S7.2. Convert user preference assessment matrix to preference cloud matrix
    C_Ex, C_En, C_He = Convert_LIT_to_CloudModel(input) # sizes of C_Ex, C_En and C_He are n×n
    
    # S7.3. Calculate the normalized preference cloud matrix
    sum_of_row = np.sum(C_Ex, axis=1)
    sum_of_col = np.sum(C_Ex, axis=0)
    s = max(max(sum_of_row),max(sum_of_col))
    C_Ex_N = C_Ex/s
    C_En_N = C_En/s
    C_He_N = C_He/s
    
    # S7.4. Calculate the total preference cloud matrix
    I = np.identity(C_Ex_N.shape[0])
    T_Ex = C_Ex_N * np.linalg.inv(I - C_Ex_N)
    T_En = C_En_N * np.linalg.inv(I - C_En_N)
    T_He = C_He_N * np.linalg.inv(I - C_He_N)
    
    # S7.5. Calculate the prominence and relation
    # calculate the sum of rows of T_Ex/T_En/T_He
    SR_Ex = np.zeros(T_Ex.shape[0])
    SR_En = np.zeros(T_En.shape[0])
    SR_He = np.zeros(T_He.shape[0])
    for i in range(T_Ex.shape[0]):
        tmp_SR_Ex = 0.
        tmp_SR_En = 0.
        tmp_SR_He = 0.
        for j in range(T_Ex.shape[1]):
            tmp_SR_Ex += T_Ex[i,j]
            tmp_SR_En += np.power(T_En[i,j],2)
            tmp_SR_He += np.power(T_He[i,j],2)
        SR_Ex[i] = tmp_SR_Ex
        SR_En[i] = np.sqrt(tmp_SR_En)
        SR_He[i] = np.sqrt(tmp_SR_He)
    # calculate the sum of columns of T_Ex/T_En/T_He
    SC_Ex = np.zeros(T_Ex.shape[1])
    SC_En = np.zeros(T_En.shape[1])
    SC_He = np.zeros(T_He.shape[1])
    for j in range(T_Ex.shape[1]):
        tmp_SC_Ex = 0.
        tmp_SC_En = 0.
        tmp_SC_He = 0.
        for i in range(T_Ex.shape[0]):
            tmp_SC_Ex += T_Ex[i,j]
            tmp_SC_En += np.power(T_En[i,j],2)
            tmp_SC_He += np.power(T_He[i,j],2)
        SC_Ex[j] = tmp_SC_Ex
        SC_En[j] = np.sqrt(tmp_SC_En)
        SC_He[j] = np.sqrt(tmp_SC_He)
    # calculate the prominence (P)
    P_Ex = np.zeros(SR_Ex.shape)
    P_En = np.zeros(SR_En.shape)
    P_He = np.zeros(SR_He.shape)
    for i in range(SC_Ex.shape[0]):
        P_Ex[i] = SR_Ex[i] + SC_Ex[i]
        P_En[i] = np.sqrt(np.power(SR_En[i],2) + np.power(SC_En[i],2))
        P_He[i] = np.sqrt(np.power(SR_He[i],2) + np.power(SC_He[i],2))
    # calculate the relation (R)
    R_Ex = np.zeros(SR_Ex.shape)
    R_En = np.zeros(SR_En.shape)
    R_He = np.zeros(SR_He.shape)
    for i in range(SC_Ex.shape[0]):
        R_Ex[i] = SR_Ex[i] - SC_Ex[i]
        R_En[i] = np.sqrt(np.power(SR_En[i],2) + np.power(SC_En[i],2))
        R_He[i] = np.sqrt(np.power(SR_He[i],2) + np.power(SC_He[i],2))
    
    # S7.6. Calculate user preference weights
    # calculate P_L and P_U
    P_L = np.zeros(P_Ex.shape)
    P_U = np.zeros(P_Ex.shape)
    P = np.around(P_Ex,4)
    for i in range(P_Ex.shape[0]):
        P_L[i] = P_Ex[i] - 3*P_En[i] - 9*P_He[i]
        P_U[i] = P_Ex[i] + 3*P_En[i] + 9*P_He[i]
    # calculate R_L and R_U
    R_L = np.zeros(R_Ex.shape)
    R_U = np.zeros(R_Ex.shape)
    R = np.around(R_Ex,4)
    for i in range(R_Ex.shape[0]):
        R_L[i] = R_Ex[i] - 3*R_En[i] - 9*R_He[i]
        R_U[i] = R_Ex[i] + 3*R_En[i] + 9*R_He[i]
        
    # calculate weight
    weights_by_Cebi = weight_by_Cebi_method(P_L,P_U)
    weights_by_Wang = weight_by_Wang_method(R_L,R_U)
    weights_by_Zhou = weight_by_Zhou_method(P_L,P_U,P_Ex,R_L,R_U,R_Ex)
    weights_by_our_method = weight_by_our_method(P_L,P_U,R_L,R_U)
    
    return [P,R],[weights_by_Cebi,weights_by_Wang,weights_by_Zhou,weights_by_our_method]

## Fuzzy based PROMETHEE II

### Preference function

#### Type 1: Usual

In [8]:
def Preference_Function_Type1(x):
    if x <= 0:
        return 0
    else:
        return 1

#### Type 2: U-shape

In [9]:
def Preference_Function_Type2(x):
    q = 0.5
    if x <= q:
        return 0
    else:
        return 1

#### Type 3: V-shape

In [10]:
def Preference_Function_Type3(x):
    p = 0.5
    if x <= 0:
        return 0
    elif x > p:
        return 1
    else:
        return x/p

#### Type 4: Level

In [11]:
def Preference_Function_Type4(x):
    q = 0.5
    p = 1
    if x <= q:
        return 0
    elif x > p:
        return 1
    else:
        return 0.5

#### Type 5: V-shape with indifference

In [12]:
def Preference_Function_Type5(x):
    q = 0.5
    p = 1
    if x <= q:
        return 0
    elif x > p:
        return 1
    else:
        return (x-q)/(x-p)

#### Type 6: Gaussian

In [13]:
def Preference_Function_Type6(x):
    s = 0.5
    if x <= 0:
        return 0
    else:
        return 1 - np.exp(-np.power(x,2)/np.power(s,2))

### Define the Calculate_PF function to calculate the preference value according to the specified function

In [14]:
def Calculate_PF(d,C_PF_type):
    if C_PF_type == 'PF_type1':
        return Preference_Function_Type1(d)
    elif C_PF_type == 'PF_type2':
        return Preference_Function_Type2(d)
    elif C_PF_type == 'PF_type3':
        return Preference_Function_Type3(d)
    elif C_PF_type == 'PF_type4':
        return Preference_Function_Type4(d)
    elif C_PF_type == 'PF_type5':
        return Preference_Function_Type5(d)
    elif C_PF_type == 'PF_type6':
        return Preference_Function_Type6(d)

### Define the LT_to_TFN function to convert the evaluation results of expert group to TFN values

In [15]:
def Convert_LT_to_TNF(language_term_matrix):
    '''
    input:
        language_term_matrix: language term assessment matrix provided by experts,
                              size: n×m
    output:
        TNF_matrix_L: lower boundary of triangular fuzzy matrix
        TNF_matrix_M: mean value of triangular fuzzy matrix
        TNF_matrix_U: upper boundary of triangular fuzzy matrix
    '''
    language_term_table = {'VL': (0,0,0.25),
                           'L':  (0,0.25,0.5),
                           'M':  (0.25,0.5,0.75),
                           'H':  (0.5,0.75,1.),
                           'VH':  (0.75,1.,1.)}
    TNF_matrix_L = np.zeros(language_term_matrix.shape)
    TNF_matrix_M = np.zeros(language_term_matrix.shape)
    TNF_matrix_U = np.zeros(language_term_matrix.shape)
    for i in range(language_term_matrix.shape[0]):
        for j in range(language_term_matrix.shape[1]):
            value_ij = language_term_table[language_term_matrix[i][j]]
            TNF_matrix_L[i][j] = value_ij[0]
            TNF_matrix_M[i][j] = value_ij[1]
            TNF_matrix_U[i][j] = value_ij[2]
    return TNF_matrix_L, TNF_matrix_M, TNF_matrix_U

### Define the Fuzzy _PROMETHEE_II function to calculate the preference weight of expert group 

In [16]:
def Fuzzy_PROMETHEE_II(A_L,A_M,A_U,C_PF_types,C_weights):
    # the size of A_L/A_M/A_U: n×m×K
    # S8.2. Calculate the expert group evaluation matrix
    B_L = np.sum(A_L,axis=0)/A_L.shape[0]
    B_M = np.sum(A_M,axis=0)/A_M.shape[0]
    B_U = np.sum(A_U,axis=0)/A_U.shape[0]
    
    # S8.4. Calculate the preference index for each demand
    '''
    存储格式(n×(n-1))为：
    a1-> a1,a2,...,an
    a2-> a1,a2,...,an
    ...
    an-> a1,a2,...,an
    '''
    PI_L_plus = np.zeros((B_L.shape[0],B_L.shape[0]))
    PI_L_minus = np.zeros((B_L.shape[0],B_L.shape[0]))
    PI_M_plus = np.zeros((B_M.shape[0],B_M.shape[0]))
    PI_M_minus = np.zeros((B_M.shape[0],B_M.shape[0]))
    PI_U_plus = np.zeros((B_U.shape[0],B_U.shape[0]))
    PI_U_minus = np.zeros((B_U.shape[0],B_U.shape[0]))
    
    for i in range(B_L.shape[0]):  # n行--取ai
        for j in range(B_L.shape[0]):  # n行--取aj
            if i == j:  #当ai==ai
                PI_L_plus[i][j] = 0
                PI_L_minus[i][j] = 0
                PI_M_plus[i][j] = 0
                PI_M_minus[i][j] = 0
                PI_U_plus[i][j] = 0
                PI_U_minus[i][j] = 0
                continue
            else: # 比较ai和aj
                PI_L_plus_ai_aj = 0.
                PI_L_minus_ai_aj = 0.
                PI_M_plus_ai_aj = 0.
                PI_M_minus_ai_aj = 0.
                PI_U_plus_ai_aj = 0.
                PI_U_minus_ai_aj = 0.
                for m in range(B_L.shape[1]):  # 计算ai和aj在m项指标下
                    # 获取准则cm的偏好函数
                    cm_PF_type = C_PF_types[m]
                    cm_weight = C_weights[m]
                    # 计算ai和aj之间的偏好值下限
                    ai_cm_L = B_L[i][m]  # value of ai_cm_L
                    aj_cm_L = B_L[j][m]  # value of aj_cm_L
                    ai_aj_L_plus = cm_weight * Calculate_PF(ai_cm_L - aj_cm_L, cm_PF_type)  # ai相对于aj的优势
                    ai_aj_L_minus = cm_weight * Calculate_PF(aj_cm_L - ai_cm_L, cm_PF_type)  # ai相对于aj的劣势
                    PI_L_plus_ai_aj += ai_aj_L_plus
                    PI_L_minus_ai_aj += ai_aj_L_minus
                    # 计算ai和aj之间的偏好值中值
                    ai_cm_M = B_M[i][m]  # value of ai_cm_M
                    aj_cm_M = B_M[j][m]  # value of aj_cm_M
                    ai_aj_M_plus = cm_weight * Calculate_PF(ai_cm_M - aj_cm_M, cm_PF_type)  # ai相对于aj的优势
                    ai_aj_M_minus = cm_weight * Calculate_PF(aj_cm_M - ai_cm_M, cm_PF_type)  # ai相对于aj的劣势
                    PI_M_plus_ai_aj += ai_aj_M_plus
                    PI_M_minus_ai_aj += ai_aj_M_minus
                    # 计算ai和aj之间的偏好值上限
                    ai_cm_U = B_U[i][m]  # value of ai_cm_U
                    aj_cm_U = B_U[j][m]  # value of aj_cm_U
                    ai_aj_U_plus = cm_weight * Calculate_PF(ai_cm_U - aj_cm_U, cm_PF_type)  # ai相对于aj的优势
                    ai_aj_U_minus = cm_weight * Calculate_PF(aj_cm_U - ai_cm_U, cm_PF_type)  # ai相对于aj的劣势
                    PI_U_plus_ai_aj += ai_aj_U_plus
                    PI_U_minus_ai_aj += ai_aj_U_minus
                # 添加到PI列表中
                PI_L_plus[i][j] = PI_L_plus_ai_aj
                PI_L_minus[i][j] =  PI_L_minus_ai_aj
                PI_M_plus[i][j] = PI_M_plus_ai_aj
                PI_M_minus[i][j] = PI_M_minus_ai_aj
                PI_U_plus[i][j] = PI_U_plus_ai_aj
                PI_U_minus[i][j] = PI_U_minus_ai_aj
    
    # S8.5. Calculate the input, output and net flows
    # calculate the input and output flow
    input_flow_L = np.sum(PI_L_plus, axis=1)
    input_flow_M = np.sum(PI_M_plus, axis=1)
    input_flow_U = np.sum(PI_U_plus, axis=1)
    output_flow_L = np.sum(PI_L_minus, axis=1)
    output_flow_M = np.sum(PI_M_minus, axis=1)
    output_flow_U = np.sum(PI_U_minus, axis=1)
    # Convert TNF to crisp value
    input_flow = (input_flow_L + 2 * input_flow_M + input_flow_U)/4
    output_flow = (output_flow_L + 2 * output_flow_M + output_flow_U)/4
    # calculate the net flow
    net_flow = input_flow - output_flow
    #print(net_flow)
    #print(np.argsort(net_flow)+1)
    net_flow += B_L.shape[0]
    # S8.6. Calculate expert preference weights based on net flows
    sum_flow = np.sum(net_flow)
    expert_preference_weights = np.around(net_flow/sum_flow,4)
    return expert_preference_weights

## Load the evaluation results of user  and calculate the preference weight of user 

In [35]:
# S7.1. Obtain the user assessment matrix
file_path = './user evaluation results.xlsx'
user_evaluation_data = pd.read_excel(file_path, index_col=0)
user_evaluation_data_np = user_evaluation_data.to_numpy()

# calculate user preference weights by Cloud_DEMATEL function
PR_value,weights_by_different_methods = Cloud_DEMATEL(user_evaluation_data_np)

# save P and R to excel file
PR = np.array(PR_value)
df_PR = pd.DataFrame(PR.T)
writer_PR = pd.ExcelWriter('PR value.xlsx')
df_PR.to_excel(writer_PR, 'PR value',float_format='%.4f',header=['P','R'],index=False)
writer_PR.save()
writer_PR.close()
# get the ranking of user preference weights obtained by different methods
weights_by_Cebi = weights_by_different_methods[0]
weights_by_Wang = weights_by_different_methods[1]
weights_by_Zhou = weights_by_different_methods[2]
weights_by_ours = weights_by_different_methods[3]
ranking_Cebi = get_ranking(weights_by_Cebi)
ranking_Wang = get_ranking(weights_by_Wang)
ranking_Zhou = get_ranking(weights_by_Zhou)
ranking_ours = get_ranking(weights_by_ours)

# Save user preference weights and ranking
header = ['Cebi(2013)','ranking','Wang(2015)','ranking','Zhou et al.(2023)','ranking','Ours','ranking']
weights_rankings = np.array([weights_by_Cebi,ranking_Cebi,weights_by_Wang,ranking_Wang,weights_by_Zhou,ranking_Zhou,weights_by_ours,ranking_ours])
df_weights_rankings = pd.DataFrame(weights_rankings.T)
writer_weights_rankings = pd.ExcelWriter('preference weights and rankings of the new user obtained by different methods.xlsx')
df_weights_rankings.to_excel(writer_weights_rankings,'weights and rankings',header=header,index=False,float_format='%.4f')
writer_weights_rankings.save()
writer_weights_rankings.close()

# save only the preference weights of the new user
header = ['Cebi(2013)','Wang(2015)','Zhou et al.(2023)','Ours']
weights = np.array([weights_by_Cebi,weights_by_Wang,weights_by_Zhou,weights_by_ours])
df_weights = pd.DataFrame(weights.T)
writer_weights = pd.ExcelWriter('preference weights of the new user obtained by different methods.xlsx')
df_weights.to_excel(writer_weights,'weights',header=header,index=False,float_format='%.4f')
writer_weights.save()
writer_weights.close()

# save only the ranking of weights
header = ['ranking_by_Cebi(2013)','ranking_by_Wang(2015)','ranking_by_Zhou et al.(2023)','ranking_by_Ours']
rankings = np.array([ranking_Cebi,ranking_Wang,ranking_Zhou,ranking_ours])
df_rankings = pd.DataFrame(rankings.T)
writer_rankings = pd.ExcelWriter('the ranking of weights obtained by different methods.xlsx')
df_rankings.to_excel(writer_rankings,'rankings',header=header,index=False)
writer_rankings.save()
writer_rankings.close()

# print the preference weights and ranking obtained by our method
print('='*60)
print('User preference weights:')
print(weights_by_ours)
print('Ascending sort index of user preference weights:')
print(ranking_ours)
print('='*60)

SR_Ex:
[0.0431 0.0393 0.0431 0.0588 0.0633 0.082  0.0145 0.023  0.0154 0.0031
 0.0116 0.0151 0.0165 0.0298 0.0332 0.029  0.02   0.0297 0.0097 0.0201
 0.0342 0.0352 0.0546 0.0029 0.0434 0.0307]
[ 6.  8.  7.  3.  2.  1. 22. 16. 20. 25. 23. 21. 19. 13. 11. 15. 18. 14.
 24. 17. 10.  9.  4. 26.  5. 12.]
SC_Ex:
[0.0179 0.0199 0.0169 0.0096 0.0066 0.0024 0.0436 0.0324 0.0454 0.0767
 0.0524 0.0425 0.0415 0.024  0.0217 0.026  0.0366 0.0249 0.0549 0.0363
 0.021  0.0193 0.009  0.0795 0.0156 0.0249]
[20. 18. 21. 23. 25. 26.  6. 11.  5.  2.  4.  7.  8. 15. 16. 12.  9. 13.
  3. 10. 17. 19. 24.  1. 22. 14.]


"\n# get the ranking of user preference weights obtained by different methods\nweights_by_Cebi = weights_by_different_methods[0]\nweights_by_Wang = weights_by_different_methods[1]\nweights_by_Zhou = weights_by_different_methods[2]\nweights_by_ours = weights_by_different_methods[3]\ndef get_ranking(input_values):\n    idx = np.argsort(-input_values)\n    ranking = np.zeros(input_values.shape)\n    for i in range(input_values.shape[0]):\n        ranking[idx[i]] = i+1\n    return ranking\nranking_Cebi = get_ranking(weights_by_Cebi)\nranking_Wang = get_ranking(weights_by_Wang)\nranking_Zhou = get_ranking(weights_by_Zhou)\nranking_ours = get_ranking(weights_by_ours)\n\n# Save user preference weights and ranking\nheader = ['Cebi(2013)','ranking','Wang(2015)','ranking','Zhou et al.(2023)','ranking','Ours','ranking']\nweights_rankings = np.array([weights_by_Cebi,ranking_Cebi,weights_by_Wang,ranking_Wang,weights_by_Zhou,ranking_Zhou,weights_by_ours,ranking_ours])\ndf_weights_rankings = pd.DataF

## Load the  evaluation results of expert group and calculate the preference weight of expert group 

In [19]:
# S8.1. Obtain (Load) the evaluation results for each expert
expert_1 = pd.read_excel('./expert group evaluation results.xlsx',sheet_name = 'expert_1',index_col=0)
expert_2 = pd.read_excel('./expert group evaluation results.xlsx',sheet_name = 'expert_2',index_col=0)
expert_3 = pd.read_excel('./expert group evaluation results.xlsx',sheet_name = 'expert_3',index_col=0)

# Convert expert evaluation lingustic term to TNF
expert_1_L,expert_1_M,expert_1_U = Convert_LT_to_TNF(expert_1.to_numpy())
expert_2_L,expert_2_M,expert_2_U = Convert_LT_to_TNF(expert_1.to_numpy())
expert_3_L,expert_3_M,expert_3_U = Convert_LT_to_TNF(expert_1.to_numpy())
expert_group_L = np.array([expert_1_L, expert_2_L, expert_3_L])
expert_group_M = np.array([expert_1_M, expert_2_M, expert_3_M])
expert_group_U = np.array([expert_1_U, expert_2_U, expert_3_U])

# S8.3. Determine the preference function for each criterion and the weights of expert
E_PF_types = ['PF_type1','PF_type6','PF_type6','PF_type3','PF_type4','PF_type1','PF_type3','PF_type3']
E_weights = np.array([0.15,0.15,0.15,0.15,0.05,0.1,0.15,0.1])

# Calculate expert preference weights by Fuzzy_PROMETHEE_II function
preference_weights_by_expert = Fuzzy_PROMETHEE_II(expert_group_L,expert_group_M,expert_group_U,E_PF_types,E_weights)

# Save the preference weights and rankings of experts
rankings_expert = get_ranking(preference_weights_by_expert)
weights_rankings_expert = np.array([preference_weights_by_expert,rankings_expert])
df_weights_rankings_expert = pd.DataFrame(weights_rankings_expert.T)
writer_expert = pd.ExcelWriter('the preference weights and rankings of expert.xlsx')
df_weights_rankings_expert.to_excel(writer_expert,'weights and ranking',header=['weights','ranking'],index=None)
writer_expert.save()
writer_expert.close()

# print the preference weights and rankings of experts
print('The preference weights of expert')
print(preference_weights_by_expert)
print('Rankings:')
print(rankings_expert)

The preference weights of expert
[0.0423 0.0542 0.0433 0.0386 0.0466 0.0518 0.034  0.0361 0.0401 0.0336
 0.0306 0.0373 0.035  0.0341 0.0481 0.0411 0.0349 0.0403 0.0289 0.0455
 0.0224 0.0347 0.0442 0.0274 0.0389 0.0359]
Rankings:
[ 8.  1.  7. 13.  4.  2. 21. 15. 11. 22. 23. 14. 17. 20.  3.  9. 18. 10.
 24.  5. 26. 19.  6. 25. 12. 16.]


## Calculate the combined preference weights and sort user requirements

In [20]:
# S9. calculate the combined preference weights
alpha = 0.5
combined_weights_05 = alpha * weights_by_ours + (1-alpha) * preference_weights_by_expert
# Sort user requirements according to W
ranking_combined_weights_05 = get_ranking(combined_weights_05)

# Save the combined preference weights and ranking
combined_weights_ranking = np.array([combined_weights_05,ranking_combined_weights_05])
df_combined = pd.DataFrame(combined_weights_ranking.T)
writer_combined = pd.ExcelWriter('The combined preference weights and ranking.xlsx')
df_combined.to_excel(writer_combined,'combined weights and ranking',header=['combined preference weight','ranking'],index=None)
writer_combined.save()
writer_combined.close()

# print the preference weight and ranking
print('Combined preference weights:')
print(combined_weights_05)
print('Ranking of the combined preference weights:')
print(ranking_combined_weights_05)

Combined preference weights:
[0.04805 0.05165 0.04855 0.056   0.0628  0.07705 0.026   0.0324  0.02965
 0.01875 0.02255 0.0281  0.0278  0.03565 0.04475 0.0387  0.02995 0.0387
 0.0205  0.0353  0.03255 0.0393  0.05615 0.0155  0.04655 0.0371 ]
Ranking of the combined preference weights:
[ 7.  5.  6.  4.  2.  1. 22. 17. 19. 25. 23. 20. 21. 14.  9. 12. 18. 11.
 24. 15. 16. 10.  3. 26.  8. 13.]


In [21]:
# Compare the combined preference weights and rankings for different preference reconciliation parameters
alphas = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
results_weights = []
results_rankings = []
results_weights_ranking = []
header_weights = []
header_rankings = []
header_weights_ranking = []
for i in range(len(alphas)):
    alpha_i = alphas[i]
    header_weights.append('alpha=' + str(alpha_i))
    header_rankings.append('ranking ' + '(alpha=' + str(alpha_i) + ')')
    header_weights_ranking.append('alpha=' + str(alpha_i))
    header_weights_ranking.append('ranking ' + '(alpha=' + str(alpha_i) + ')')
    # calculate the combined preference weights when alpha equals alpha_i
    combined_weights_i = alpha_i * weights_by_ours + (1-alpha_i) * preference_weights_by_expert
    ranking_combined_weights_i = get_ranking(combined_weights_i)
    results_weights.append(np.array(combined_weights_i))
    results_rankings.append(np.array(ranking_combined_weights_i))
    results_weights_ranking.append(np.array(combined_weights_i))
    results_weights_ranking.append(np.array(ranking_combined_weights_i))
# Save only the results of combined weights
df_results_weights = pd.DataFrame(np.array(results_weights).T)
df_results_weights_writer = pd.ExcelWriter('results of combined preference weights with different alpha values.xlsx')
df_results_weights.to_excel(df_results_weights_writer,'combined weights',header=header_weights,index=False)
df_results_weights_writer.save()
df_results_weights_writer.close()
# Save only the results of rankings
df_results_rankings = pd.DataFrame(np.array(results_rankings).T)
df_results_rankings_writer = pd.ExcelWriter('results of the rankings of combined preference weights with different alpha values.xlsx')
df_results_rankings.to_excel(df_results_rankings_writer,'rankings',header=header_rankings,index=False)
df_results_rankings_writer.save()
df_results_rankings_writer.close()
# Save the combined weights and ranking
df_results_weights_rankings = pd.DataFrame(np.array(results_weights_ranking).T)
df_results_weights_rankings_writer = pd.ExcelWriter('results of the combined preference weights and rankings with different alpha values.xlsx')
df_results_weights_rankings.to_excel(df_results_weights_rankings_writer, 'combined weights and rankings', header=header_weights_ranking, index=False)
df_results_weights_rankings_writer.save()
df_results_weights_rankings_writer.close()
print('Finish!')

Finish!
